In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers datasets evaluate rouge_score
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
ERROR: Operation cancelled by user
  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import evaluate
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer

In [ ]:
billsum = load_dataset("scientific_papers", "arxiv", split= "validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warn

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [ ]:
billsum

Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 6436
})

In [ ]:
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define functions for operations
def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = ' '.join(token.text for token in nlp(text) if not token.is_punct)  # Punctuation Handling
    text = ' '.join(text.split())  # Whitespace Handling
    text = ' '.join(token.text for token in nlp(text) if token.text not in STOP_WORDS)  # Stop Word Removal
    text = ' '.join(ent.text if ent.label_ != 'O' else ' '.join(token.text for token in ent) for ent in nlp(text).ents)  # NER
    return text
processed_billsum = []
# Apply operations to each dictionary in the list
for row in billsum:
    row['article'] = preprocess_text(row['article'])
    row['abstract'] = preprocess_text(row['abstract'])
    row['section_names'] = preprocess_text(row['section_names'])

In [ ]:
row['article']

'spin1/2 heisenberg half anderson nineteen long wavelength second zero 18 26 4 sixteen 16 quantum sec 3b 1 2 4 2 32 zero 2 @xmath37 20 1 0 linear 4 4 576 36 80 approximately 30 90 zero @xmath52 @xmath53 @xmath54 zero cray 64 36 5 2 ea15ad cray2 2 20 40 3 hours @xcite @xmath77 quantum monte @xmath79 @xmath101 zero second @xmath104 20 32 16 16 @xmath111 @xmath113 @xmath114 precise value second @xmath115 @xmath112 @xmath118 linear linear zero @xmath128 @xmath129 @xmath130 @xmath131 @xmath12 @xmath133 @xmath129 second @xmath136 2 percent @xmath144 quantum @xmath146 @xmath147 @xmath30 cluster @xmath20 @xmath149 second eqs.([fsm @xmath30 @xmath156 @xmath157 @xmath112 @xmath162 monte carlo @xmath163 @xmath164 31561400 16 @xmath169 half 2 @xmath172 @xmath174 @xmath176 @xmath177 @xmath182 @xmath184 spin m0sw m1sw @xmath131 @xmath186 @xmath187 @xmath188 @xmath189 @xmath191 heisenberg model square second @xmath131 @xmath30 cluster second @xmath30 cluster @xmath196 @xmath198 second small@xmath199 

In [ ]:
def segment_sentences(text):
    doc = nlp(text)
    return [sent.text for sent in doc.sents]

# Apply sentence segmentation to each dictionary in the list
for row in billsum:
    row['article'] = segment_sentences(row['article'])
    row['abstract'] = segment_sentences(row['abstract'])
    row['section_names'] = segment_sentences(row['section_names'])

In [ ]:
row['article'][1]

'the notations @xmath10 and @xmath11 indicate summation over the nearest and next \nnearest neighbor bonds on a square lattice , each bond being counted once .'

In [ ]:
billsum['article'][100]

'silicon carbide is a wide - band - gap semiconductor material especially suitable for high power , high frequency and high temperature applications .\na common technique for the incorporation of dopant atoms into sic is ion implantation .\nthe post - implantation annealing is needed to reduce the damage . at the same time\n, the thermal treatment creates new stable defect centers which influence the material properties .\nthe persistent centers that tend to grow even at annealing temperatures above 1000@xmath3c are e.g.  the photoluminescence ( pl ) centers d@xmath4 ( ref .  ) and d@xmath1 ( ref .  ) .\nmost likely , the formation of these centers is related to the aggregation of the intrinsic defects .\nfor example , vacancy or antisite clusters have been discussed@xcite in conjunction with the d@xmath4-center .\nthe d@xmath1-center , with its rich vibrational spectrum located above the sic phonon spectrum , is considered as a carbon - related defect.@xcite up to now , the complexity

In [ ]:
# Define function for lemmatization
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = ' '.join(token.lemma_ for token in doc)
    return lemmatized_text
# Load pre-trained word embeddings (example)
def load_embeddings():
    # Load pre-trained word embeddings (example)
    embeddings = {}
    # Load embeddings from file or using a library like gensim
    return embeddings

# Function to obtain domain-specific embeddings for scientific terms
def get_domain_embeddings(text, embeddings):
    domain_embeddings = []
    for term in text.split():
        if term in embeddings:
            domain_embeddings.append(embeddings[term])
        else:
            # Handle out-of-vocabulary terms (optional)
            # You can ignore or use a default embedding
            domain_embeddings.append(np.zeros(embedding_dim))  # Example: Zero vector
    return domain_embeddings
# Load pre-trained word embeddings
embeddings = load_embeddings()

# Define embedding dimensionality
embedding_dim = 300  # Example: dimensionality of 300 for word embedding


# Apply preprocessing and obtain domain-specific embeddings for each text column
for row in billsum:
    row['article'] = lemmatize_text(row['article'])
    row['abstract'] = lemmatize_text(row['abstract'])
    row['section_names'] = lemmatize_text(row['section_names'])
    row['article_embeddings'] = get_domain_embeddings(row['article'], embeddings)
    row['abstract_embeddings'] = get_domain_embeddings(row['abstract'], embeddings)
    row['section_names_embeddings'] = get_domain_embeddings(row['section_names'], embeddings)

In [ ]:
checkpoint = "allenai/led-base-16384"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, padding="max_length", max_length=4096, truncation=True)

    labels = tokenizer(text_target=examples["abstract"], padding="max_length", max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = dataset.map(preprocess_function, batched=True,remove_columns=["article", "abstract", "section_names"],)

Map:   0%|          | 0/5148 [00:00<?, ? examples/s]

Map:   0%|          | 0/1288 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum

NameError: name 'tokenized_billsum' is not defined

In [ ]:
tokenized_billsum.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)


rouge = evaluate.load("rouge")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_sci_model",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    predict_with_generate=True,
    fp16="True",
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    push_to_hub=True,
)

In [ ]:
tokenized_billsum

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5148
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1288
    })
})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
)

NameError: name 'model' is not defined

In [ ]:
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
dataset = billsum.train_test_split(test_size=0.2)

In [ ]:
checkpoint = "philschmid/bart-large-cnn-samsum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenized_billsum

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5148
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1288
    })
})

In [ ]:


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)


rouge = evaluate.load("rouge")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_sci_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

In [ ]:
tokenized_billsum

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5148
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1288
    })
})

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


billsum


In [ ]:
billsum

Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 6436
})

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.529600,2.467935,0.431300,0.152700,0.251400,0.357400,131.801200
2,2.175800,2.401726,0.430600,0.150200,0.250700,0.359300,123.674700
3,1.925600,2.418611,0.434800,0.153100,0.255100,0.361200,123.750800
4,1.721400,2.464087,0.430500,0.148600,0.248700,0.356900,129.313700
5,1.590500,2.505140,0.435200,0.150800,0.251600,0.360300,128.508500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=8580, training_loss=2.003287248233537, metrics={'train_runtime': 24326.2359, 'train_samples_per_second': 1.058, 'train_steps_per_second': 0.353, 'total_flos': 5.578127245836288e+16, 'train_loss': 2.003287248233537, 'epoch': 5.0})